# Main Operations with LCEL Chains

## Contents
* Chaining runnables, coercion
* Multiple chains
* Nested chains
* Fallback for chains

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [4]:
!pip list | grep langchain

langchain                 0.2.3
langchain-community       0.2.4
langchain-core            0.2.5
langchain-openai          0.1.8
langchain-text-splitters  0.2.1

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


## Connect with LLM and start conversation with it

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

## Basic chain: LLM model + prompt + output parser

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Write one brief sentence about {politician}")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [8]:
chain.invoke({
    "politician": "JFK"
})

'John F. Kennedy was the 35th President of the United States, serving from 1961 until his assassination in 1963.'

## Mid-level chain: Retriever App Example

In [9]:
#!pip install docarray tiktoken

In [10]:
!pip list | grep tiktoken

tiktoken                  0.7.0

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [11]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["AI Accelera has provided Generative AI Training and Consulting Services in more than 100 countries", "Aceleradora AI is the branch of AI Accelera for the Spanish-Speaking market"],
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

output_parser = StrOutputParser()

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv061224-p1/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [12]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

get_question_and_retrieve_relevant_docs = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

* The previous code creates a `RunnableParallel` object with two entries.
    * The first entry, context, will include the document results fetched by the retriever.
    * The second entry, question, will contain the user’s original question. To pass on the question, we use `RunnablePassthrough` to copy this entry.

In [13]:
chain = get_question_and_retrieve_relevant_docs | prompt | model | output_parser

chain.invoke("In how many countries has AI Accelera provided services?")

'AI Accelera has provided services in more than 100 countries.'

* Graphic representation of what this chain does:

![alt text](LCEL-chain-graph.png)

## RunnableParallel

In [14]:
#!pip install faiss-cpu

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 7.000 Alumni from all continents and top companies"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("who are the Alumni of AI Accelera?")

'The Alumni of AI Accelera are individuals from all continents and top companies, totaling more than 7,000 trained.'

### Rememeber: Syntax of RunnableParallel can have several variations
* When composing RunnableParallel with another Runnable we do not need to wrap it up in RunnableParallel class. Inside chain, next three syntaxs are equivalent:
    * `RunnableParallel({"context": retriever, "question": RunnablePassthrough()})`
    * `RunnableParallel(context=retriever, question=RunnablePassthrough())`
    * `{"context": retriever, "question": RunnablePassthrough()}`

## Using itemgetter with RunnableParallel

In [16]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(model="gpt-3.5-turbo")

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 3,000 Enterprise Alumni."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "How many Enterprise Alumni has trained AI Accelera?", "language": "Pirate English"})

'Arrr, AI Accelera has trained more than 3,000 Enterprise Alumni.'

## RunnablePassthrough

In [17]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    user_input = RunnablePassthrough(),
    transformed_output = lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'user_input': {'num': 1}, 'transformed_output': 2}

## Chaining Runnables

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

model = ChatOpenAI(model="gpt-3.5-turbo")

chain = prompt | model | StrOutputParser()

In [19]:
chain.invoke("Chamberlain")

'Chamberlain was a British politician who is best known for his policy of appeasement towards Nazi Germany in the years leading up to World War II.'

### Coercion: Combine chain (which is Runnable) with other Runnables to create new chain

In [20]:
from langchain_core.output_parsers import StrOutputParser

historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [21]:
composed_chain.invoke({"politician": "Lincoln"})

"Yes, Abraham Lincoln is considered to have had a positive impact on humanity. His leadership during the Civil War helped to preserve the unity of the United States and ultimately led to the end of slavery. The Emancipation Proclamation was a significant step towards equality and justice for all individuals, and Lincoln's efforts to abolish slavery have had a lasting impact on society. Additionally, Lincoln's leadership and dedication to upholding the principles of democracy and freedom have inspired generations of Americans and individuals around the world."

In [22]:
composed_chain.invoke({"politician": "Attila"})

"Attila the Hun's conquests and actions were not necessarily positive for humanity. He was known for his brutal tactics, including pillaging and destroying cities, and his reign brought suffering and devastation to many people. While he was certainly a powerful and formidable leader, his legacy is more often associated with violence and destruction rather than positive contributions to humanity."

### Functions can also be included in Runnables

In [23]:
composed_chain_with_lambda = (
    chain
    | (lambda input: {"politician": input})
    | historian_prompt
    | model
    | StrOutputParser()
)

In [24]:
composed_chain_with_lambda.invoke({"politician": "Robespierre"})

"It is a matter of debate whether Robespierre's actions during the Reign of Terror were ultimately positive for humanity. While he did play a key role in the French Revolution and advocated for radical social and political change, his methods were often brutal and oppressive. The Reign of Terror led to the deaths of thousands of people, many of whom were executed without fair trial. Some argue that Robespierre's actions were necessary to protect the revolution and establish a new order, while others believe that the violence and repression of the Reign of Terror ultimately did more harm than good. Ultimately, the question of whether Robespierre was a positive figure for humanity is a complex and contentious one."

## Multiple chains

In [25]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician": "Miterrand", "language": "French"})

"Le continent où se trouve la France, dont François Mitterrand était originaire, est l'Europe."

## Nested chains

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableParallel

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

chain = RunnableParallel(
    {
        "soccer_player": RunnablePassthrough() 
        | RunnableLambda(russian_lastname_from_dictionary), 
        "operation_c": RunnablePassthrough()
    }
) | prompt | model | output_parser

In [27]:
chain.invoke({
    "name1": "Jordam",
    "name": "Abram"
})

"One curious fact about Roman Abramovich is that he is known to be a collector of rare and expensive art pieces. He has an extensive art collection that includes works by renowned artists such as Francis Bacon, Lucian Freud, and Andy Warhol. Abramovich's love for art is reflected in the design of his luxury yacht, which features a collection of expensive artworks displayed throughout the vessel."

## Fallback for chains

In [6]:
# First let's create a chain with a ChatModel
# We add in a string output parser here so the outputs between the two are the same type
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a funny assistant who always includes a joke in your response",
        ),
        ("human", "Who is the best {sport} player worldwide?"),
    ]
)
# Here we're going to use a bad model name to easily create a chain that will error
chat_model = ChatOpenAI(model="gpt-fake")

bad_chain = chat_prompt | chat_model | StrOutputParser()

In [7]:
# Now lets create a chain with the normal OpenAI model
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

prompt_template = """Instructions: You're a funny assistant who always includes a joke in your response.

Question: Who is the best {sport} player worldwide?"""

prompt = PromptTemplate.from_template(prompt_template)

llm = OpenAI()

good_chain = prompt | llm

In [8]:
# We can now create a final chain which combines the two
chain = bad_chain.with_fallbacks([good_chain])

chain.invoke({"sport": "soccer"})

"\n\nResponse: Well, it depends on who you ask. Some might say Messi, others might say Ronaldo. But I personally think it's my grandma, she can kick a ball farther than anyone I know!"